# 교차 검증과 그리드 서치 (하이퍼 파라미터 튜닝하기)

> 교차 검증과 그리드 서치를 이용해서 결정 트리의 최적의 하이퍼 파라미터를 찾는다.

* 사용한 모델 : 결정 트리

여러 하이퍼 파라미터의 조합으로 모델을 만들어서 테스트 세트로 성능을 평가하면 결국 테스트 세트에 모델을 맞추는 것이다.

즉, 일반화 성능을 테스트 세트를 통해서 제대로 측정하기 위해서는 테스트 세트를 마지막에 한 번만 사용해서 성능을 측정해야 한다.

최적의 하이퍼 파라미터 조합을 찾기 위해서 모델의 성능 평가를 테스트 세트가 아닌 훈련 세트의 일부분을 사용해야 하는데 이 데이터를 '검증 세트'라고 한다.

> 검증 세트 (validation set): 모델의 성능을 평가하기 위해 훈련 세트에서 일부분 뗴어낸 데이터

이번 장에서는 검증 세트와 교차 검증에 대해서 알아보고, 최적의 하이퍼 파라미터를 찾는 방법인 그리드 서치와 랜덤 서치에 대해서 알아본다.

## 검증 세트 (validation set)

### 데이터 준비

In [1]:
import pandas as pd
wine = pd.read_csv("https://bit.ly/wine_csv_data")
wine.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6497 entries, 0 to 6496
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   alcohol  6497 non-null   float64
 1   sugar    6497 non-null   float64
 2   pH       6497 non-null   float64
 3   class    6497 non-null   float64
dtypes: float64(4)
memory usage: 203.2 KB


In [2]:
data = wine[["alcohol", "sugar", "pH"]].to_numpy()
target = wine["class"].to_numpy()

print(data.shape, target.shape)

(6497, 3) (6497,)


In [3]:
from sklearn.model_selection import train_test_split

train_input, test_input, train_target, test_target = train_test_split(
    data, target, test_size=0.2, random_state=42
)

print(train_input.shape, train_target.shape)
print(test_input.shape, test_target.shape)

(5197, 3) (5197,)
(1300, 3) (1300,)


In [4]:
# train_input과  train_target을 다시 훈련 세트(sub)와 검증 세트(val)로 나눈다.
sub_input, val_input, sub_target, val_target = train_test_split(
    train_input, train_target, test_size=0.2, random_state=42
)

print(sub_input.shape, sub_target.shape)
print(val_input.shape, val_target.shape)

(4157, 3) (4157,)
(1040, 3) (1040,)


### 모델 생성 및 검증

In [5]:
from sklearn.tree import DecisionTreeClassifier

dt = DecisionTreeClassifier(random_state=42)
dt.fit(sub_input, sub_target)

# 이 모델은 훈련 세트에 과대적합 되어있으므로 하이퍼 파라미터를 바꾸면서 더 좋은 모델을 찾아야 한다.
print(dt.score(sub_input, sub_target)) # 훈련 세트로 모델을 훈련
print(dt.score(val_input, val_target)) # 검증 세트로 모델을 평가

0.9971133028626413
0.864423076923077


## 교차 검증 (cross validation)

검증 세트를 뗴어내면 훈련 세트의 크기가 작아진다.

일반적으로 훈련 세트의 크기가 클 수록 모델의 훈련이 잘 되지만, 검증 세트의 크기가 작아져서 검증 점수를 신뢰할 수 없게 된다.

그렇다고 검증 세트의 크기를 너무 크게하면 모델의 훈련을 잘 할 수 없게 된다.

이러한 문제점을 해결하기 위해 교차 검증을 수행할 수 있는데, 교차 검증이란 검증 세트를 만들어서 모델을 평가하는 것을 여러 번 수행해서 점수들의 평균을 최종 점수로 얻는 방식이다.

> k-fold cross validation: 훈련 세트를 k개의 폴드로 분할해서 교차 검증을 수행

교차 검증을 수행하면 훈련 세트를 충분히 확보하면서도 평균 점수를 최종 점수로 얻기 때문에 검증 점수도 안정적으로 확보할 수 있다.

In [6]:
# 사이킷런의 교차 검증을 수행하는 함수 (기본 5-fold cross validation)
from sklearn.model_selection import cross_validate

scores = cross_validate(dt, train_input, train_target)
print(scores)

{'fit_time': array([0.00430584, 0.00420403, 0.00416207, 0.00397205, 0.00381494]), 'score_time': array([0.00033808, 0.00036407, 0.00022888, 0.00022984, 0.00022125]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}


In [7]:
# 교차 검증의 최종 점수
import numpy as np

print(np.mean(scores["test_score"]))

0.855300214703487


In [8]:
# cross_validate() 함수는 훈련 세트를 섞어서 폴드를 나누지 않는다.
# train_test_split() 함수로 전체 데이터를 섞은 후 훈련 세트를 만들었기 떄문에 섞을 필요가 없지만
# 만약 데이터를 섞어서 교차 검증을 수행해야 한다면 '분할기'를 지정해야 한다.
from sklearn.model_selection import StratifiedKFold

splitter = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
scores = cross_validate(dt, train_input, train_target, cv=splitter)
print(np.mean(scores["test_score"]))

0.8574181117533719


## 하이퍼 파라미터 튜닝

하이퍼 파라미터는 모델이 학습할 수 없어서 사용자가 직접 지정해야하는 파라미터이다.

먼저 기본값으로 모델을 훈련하고, 값을 바꿔 가면서 교차 검증을 수행해서 최적의 하이퍼 파라미터의 조합을 찾아야 한다.

하지만 일반적으로 하이퍼 파라미터는 여러 개의 파라미터를 동시에 바꿔가면서 확인해야 하기 때문에 사람이 직접 모두 다 테스트하는 것은 쉽지 않다.

따라서 그리드 서치와 같이 사이킷런에서 제공하는 도구를 사용하는 것이 바람직하다.

사이킷런에서는 하이퍼 파라미터 탐색을 위한 도구로 '그리드 서치'와 '랜덤 서치'를 제공한다.

In [9]:
# 그리드 서치
from sklearn.model_selection import GridSearchCV

params = {"min_impurity_decrease": [0.0001, 0.0002, 0.0003, 0.0004, 0.0005]}
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'min_impurity_decrease': [0.0001, 0.0002, 0.0003,
                                                   0.0004, 0.0005]})

In [10]:
# 검증 점수가 가장 높은 모델 (전체 훈련 세트로부터 훈련을 마친 상태)
dt = gs.best_estimator_
print(dt.score(train_input, train_target))

0.9615162593804117


In [11]:
# 그리드 서치로 찾은 최적의 하이퍼 파라미터 조합
print(gs.best_params_)

{'min_impurity_decrease': 0.0001}


In [12]:
# 각 매개변수에서 수행한 교차 검증의 평균 점수
print(gs.cv_results_["mean_test_score"])

[0.86819297 0.86453617 0.86492226 0.86780891 0.86761605]


In [13]:
# 복잡한 매개변수 탐색
params = {
    "min_impurity_decrease": np.arange(0.0001, 0.001, 0.0001),
    "max_depth": range(5, 20, 1),
    "min_samples_split": range(2, 100, 10)
}

gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': range(5, 20),
                         'min_impurity_decrease': array([0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007, 0.0008,
       0.0009]),
                         'min_samples_split': range(2, 100, 10)})

In [14]:
print(gs.best_estimator_)

DecisionTreeClassifier(max_depth=14, min_impurity_decrease=0.0004,
                       min_samples_split=12, random_state=42)


In [15]:
print(gs.cv_results_["mean_test_score"])

[0.85780355 0.85799604 0.85799604 ... 0.86126601 0.86165063 0.86357629]


In [16]:
# 랜덤 서치
# 매개변수의 값이 수치일 때 값의 범위나 간격을 미리 정하기 어려울 수 있다.
# 랜덤 서치는 매개변수 값의 목록을 전달하지 않고 매개변수를 샘플링할 수 있는 확률 분포 객체를 전달한다.
from scipy.stats import randint, uniform

params = {
    "min_impurity_decrease": uniform(0.0001, 0.001),
    "max_depth": randint(20, 50),
    "min_samples_split": randint(2, 25),
    "min_samples_leaf": randint(1, 25)
}

from sklearn.model_selection import RandomizedSearchCV

rs = RandomizedSearchCV(DecisionTreeClassifier(random_state=42), params, n_iter=100, n_jobs=-1, random_state=42)
rs.fit(train_input, train_target)

RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=42),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x1659f0ca0>,
                                        'min_impurity_decrease': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x165ac7f10>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x1659f0d90>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x165c4a5e0>},
                   random_state=42)

In [17]:
# 최적의 매개변수 조합
print(rs.best_params_)

{'max_depth': 39, 'min_impurity_decrease': 0.00034102546602601173, 'min_samples_leaf': 7, 'min_samples_split': 13}


In [18]:
# 최고의 교차 검증 점수
print(np.max(rs.cv_results_["mean_test_score"]))

0.8695428296438884


In [19]:
# 최적의 모델
dt = rs.best_estimator_
print(dt.score(test_input, test_target)) # 테스트 세트 점수

0.86
